# Automatic generation of Notebook using PyCropML
    This notebook implements a crop model.

### Model Shootnumber

In [ ]:
MODULE list_sub
    IMPLICIT NONE
    TYPE container
        CLASS(*), ALLOCATABLE :: item
        CLASS(*), ALLOCATABLE :: items(:)
    END TYPE

    interface Add
        module procedure AddToListFloat
        module procedure AddToListInt
        module procedure AddToListChar
        module procedure AddToListIntArray
    end interface
CONTAINS

    FUNCTION indice(vectorElem, elem)
        CHARACTER(LEN=*), DIMENSION(:):: vectorElem
        INTEGER::iterator, indice
        CHARACTER(LEN=*):: elem
        DO iterator= 1, SIZE(vectorElem)
            IF(vectorElem(iterator)==elem) THEN
                indice = iterator
            END IF
        END DO
        RETURN
    END FUNCTION indice


    SUBROUTINE AddToListFloat(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        REAL, INTENT(IN) :: element
        REAL, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        REAL, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListFloat

    SUBROUTINE AddToListInt(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize
        INTEGER, INTENT(IN) :: element
        INTEGER, DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        INTEGER, DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            ALLOCATE(list(1))
            list(1) = element
        END IF
    END SUBROUTINE AddToListInt

    SUBROUTINE AddToListChar(list, element)
        IMPLICIT NONE
        INTEGER :: i, isize, l
        CHARACTER(LEN=*), INTENT(IN) :: element
        CHARACTER(LEN=*), DIMENSION(:), ALLOCATABLE, INTENT(INOUT) :: list
        CHARACTER(LEN=65), DIMENSION(:), ALLOCATABLE :: clist
        IF(ALLOCATED(list)) THEN
            isize = SIZE(list)
            ALLOCATE(clist(isize+1))
            DO i=1,isize
                clist(i) = list(i)
            END DO
            clist(isize+1) = element
            DEALLOCATE(list)
            CALL MOVE_ALLOC(clist, list)
        ELSE
            l=1
            ALLOCATE(list(l))
            list(l) = element
        END IF
    END SUBROUTINE AddToListChar


    SUBROUTINE AddToListIntArray(a, e)
        INTEGER, ALLOCATABLE, INTENT(INOUT) :: a(:)
        INTEGER, ALLOCATABLE, INTENT(IN) :: e(:)
        INTEGER, ALLOCATABLE:: tmp(:)

        IF (.NOT.ALLOCATED(a)) THEN
            ALLOCATE(a(1))
            ALLOCATE(a(SIZE(e)), source = e)
        ELSE
            CALL MOVE_ALLOC(a,tmp)
            ALLOCATE(a(SIZE(tmp)+SIZE(e)), source = e)
            a(1:SIZE(tmp)) = tmp
            a(SIZE(tmp)+1:SIZE(a)) = e
        END IF
    END SUBROUTINE AddToListIntArray

END MODULE list_sub
MODULE Shootnumbermod
    USE list_sub
    IMPLICIT NONE
CONTAINS

    SUBROUTINE model_shootnumber(canopyShootNumber_t1, &
        leafNumber, &
        sowingDensity, &
        targetFertileShoot, &
        tilleringProfile_t1, &
        leafTillerNumberArray_t1, &
        numberTillerCohort_t1, &
        averageShootNumberPerPlant, &
        canopyShootNumber, &
        leafTillerNumberArray, &
        tilleringProfile, &
        numberTillerCohort)
        IMPLICIT NONE
        REAL, INTENT(IN) :: canopyShootNumber_t1
        REAL, INTENT(IN) :: leafNumber
        REAL, INTENT(IN) :: sowingDensity
        REAL, INTENT(IN) :: targetFertileShoot
        REAL, ALLOCATABLE , DIMENSION(:), INTENT(IN) :: tilleringProfile_t1
        INTEGER, ALLOCATABLE , DIMENSION(:), INTENT(IN) ::  &
                leafTillerNumberArray_t1
        INTEGER, INTENT(IN) :: numberTillerCohort_t1
        REAL, INTENT(OUT) :: averageShootNumberPerPlant
        REAL, INTENT(OUT) :: canopyShootNumber
        INTEGER, ALLOCATABLE , DIMENSION(:), INTENT(OUT) ::  &
                leafTillerNumberArray
        REAL, ALLOCATABLE , DIMENSION(:), INTENT(OUT) :: tilleringProfile
        INTEGER, INTENT(OUT) :: numberTillerCohort
        INTEGER:: emergedLeaves
        INTEGER:: shoots
        INTEGER:: i
        INTEGER, ALLOCATABLE , DIMENSION(:):: lNumberArray_rate
        !- Description:
    !            * Title: CalculateShootNumber Model
    !            * Author: Pierre MARTRE
    !            * Reference: Modeling development phase in the 
    !                Wheat Simulation Model SiriusQuality.
    !                See documentation at http://www1.clermont.inra.fr/siriusquality/?page_id=427
    !            * Institution: INRA/LEPSE Montpellier
    !            * Abstract: calculate the shoot number and update the related variables if needed
        !- inputs:
    !            * name: canopyShootNumber_t1
    !                          ** description : shoot number for the whole canopy
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLE
    !                          ** min : 0
    !                          ** max : 10000
    !                          ** default : 288.0
    !                          ** unit : shoot m-2
    !                          ** inputtype : variable
    !            * name: leafNumber
    !                          ** description : Leaf number 
    !                          ** variablecategory : state
    !                          ** inputtype : variable
    !                          ** datatype : DOUBLE
    !                          ** min : 0
    !                          ** max : 10000
    !                          ** default : 3.34
    !                          ** unit : leaf
    !            * name: sowingDensity
    !                          ** description : number of plant /m²
    !                          ** parametercategory : species
    !                          ** datatype : DOUBLE
    !                          ** min : 0
    !                          ** max : 500
    !                          ** default : 288.0
    !                          ** unit : plant m-2
    !                          ** inputtype : parameter
    !            * name: targetFertileShoot
    !                          ** description : max value of shoot number for the canopy
    !                          ** parametercategory : species
    !                          ** datatype : DOUBLE
    !                          ** min : 280
    !                          ** max : 1000
    !                          ** default : 600.0
    !                          ** unit : shoot
    !                          ** inputtype : variable
    !            * name: tilleringProfile_t1
    !                          ** description :  store the amount of new tiller created at each time a new tiller appears
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLELIST
    !                          ** default : [288.0]
    !                          ** unit : 
    !                          ** inputtype : variable
    !            * name: leafTillerNumberArray_t1
    !                          ** description : store the number of tiller for each leaf layer
    !                          ** variablecategory : state
    !                          ** datatype : INTLIST
    !                          ** unit : leaf
    !                          ** default : [1, 1, 1]
    !                          ** inputtype : variable
    !            * name: numberTillerCohort_t1
    !                          ** description :  Number of tiller which appears
    !                          ** variablecategory : state
    !                          ** datatype : INT
    !                          ** min : 0
    !                          ** max : 10000
    !                          ** default : 1
    !                          ** unit : 
    !                          ** inputtype : variable
        !- outputs:
    !            * name: averageShootNumberPerPlant
    !                          ** description : average shoot number per plant in the canopy
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLE
    !                          ** min : 0
    !                          ** max : 10000
    !                          ** unit : shoot m-2
    !            * name: canopyShootNumber
    !                          ** description : shoot number for the whole canopy
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLE
    !                          ** min : 0
    !                          ** max : 10000
    !                          ** unit : shoot m-2
    !            * name: leafTillerNumberArray
    !                          ** description : store the number of tiller for each leaf layer
    !                          ** variablecategory : state
    !                          ** datatype : INTLIST
    !                          ** unit : leaf
    !            * name: tilleringProfile
    !                          ** description :  store the amount of new tiller created at each time a new tiller appears
    !                          ** variablecategory : state
    !                          ** datatype : DOUBLELIST
    !                          ** unit : dimensionless
    !            * name: numberTillerCohort
    !                          ** description : Number of tiller which appears
    !                          ** variablecategory : state
    !                          ** datatype : INT
    !                          ** min : 0
    !                          ** max : 10000
    !                          ** unit : dimensionless
        emergedLeaves = MAX(1, CEILING(leafNumber - 1.0))
        shoots = fibonacci(emergedLeaves)
        canopyShootNumber = min(shoots * sowingDensity, targetFertileShoot)
        averageShootNumberPerPlant = canopyShootNumber / sowingDensity
        IF(canopyShootNumber .NE. canopyShootNumber_t1) THEN
            tilleringProfile = tilleringProfile_t1
            call Add(tilleringProfile,canopyShootNumber - canopyShootNumber_t1)
        END IF
        numberTillerCohort = SIZE(tilleringProfile)
        DO i = SIZE(leafTillerNumberArray_t1) , CEILING(leafNumber)-1, 1
            call Add(lNumberArray_rate, numberTillerCohort)
        END DO
        leafTillerNumberArray = leafTillerNumberArray_t1
        call Add(leafTillerNumberArray,lNumberArray_rate)
    END SUBROUTINE model_shootnumber

    RECURSIVE FUNCTION fibonacci(n) RESULT(res_cyml)
        IMPLICIT NONE
        INTEGER, INTENT(IN) :: n
        INTEGER:: res_cyml
        IF(n .LE. 1) THEN
            res_cyml = n
        ELSE
            res_cyml = fibonacci(n - 1) + fibonacci(n - 2)
        END IF
    END FUNCTION fibonacci

    SUBROUTINE init_shootnumber(sowingDensity, &
        targetFertileShoot, &
        averageShootNumberPerPlant, &
        canopyShootNumber, &
        leafTillerNumberArray, &
        tilleringProfile, &
        numberTillerCohort)
        IMPLICIT NONE
        REAL, INTENT(IN) :: sowingDensity
        REAL, INTENT(IN) :: targetFertileShoot
        REAL:: canopyShootNumber_t1
        REAL:: leafNumber
        REAL, ALLOCATABLE , DIMENSION(:):: tilleringProfile_t1
        INTEGER, ALLOCATABLE , DIMENSION(:):: leafTillerNumberArray_t1
        INTEGER:: numberTillerCohort_t1
        REAL, INTENT(OUT) :: averageShootNumberPerPlant
        REAL, INTENT(OUT) :: canopyShootNumber
        INTEGER, ALLOCATABLE , DIMENSION(:), INTENT(OUT) ::  &
                leafTillerNumberArray
        REAL, ALLOCATABLE , DIMENSION(:), INTENT(OUT) :: tilleringProfile
        INTEGER, INTENT(OUT) :: numberTillerCohort
        canopyShootNumber = sowingDensity
        averageShootNumberPerPlant = 1.0
        call Add(tilleringProfile, sowingDensity)
        numberTillerCohort = 1
        deallocate(leafTillerNumberArray)

    END SUBROUTINE init_shootnumber

END MODULE
PROGRAM test
    USE Shootnumbermod
    REAL:: canopyShootNumber_t1
    REAL:: leafNumber
    REAL:: sowingDensity
    REAL:: targetFertileShoot
    REAL, ALLOCATABLE, DIMENSION(:):: tilleringProfile_t1
    INTEGER, ALLOCATABLE, DIMENSION(:):: leafTillerNumberArray_t1
    INTEGER:: numberTillerCohort_t1
    REAL:: averageShootNumberPerPlant
    REAL:: canopyShootNumber
    INTEGER, ALLOCATABLE, DIMENSION(:):: leafTillerNumberArray
    REAL, ALLOCATABLE, DIMENSION(:):: tilleringProfile
    INTEGER:: numberTillerCohort
    print *, "--------test_test_wheat1_ShootNumber-------"
    targetFertileShoot = 600.0
    sowingDensity = 288.0
    canopyShootNumber_t1 = 288.0
    leafNumber = 3.34348137255
    leafTillerNumberArray_t1 = [1, 1, 1]
    tilleringProfile_t1 = [288.0]
    numberTillerCohort_t1 = 1
    call model_shootnumber(canopyShootNumber_t1, leafNumber,  &
            sowingDensity, targetFertileShoot, tilleringProfile_t1,  &
            leafTillerNumberArray_t1, numberTillerCohort_t1,  &
            averageShootNumberPerPlant, canopyShootNumber, leafTillerNumberArray,  &
            tilleringProfile, numberTillerCohort)
    !averageShootNumberPerPlant: 2
    print *, "averageShootNumberPerPlant estimated :"
    print *, averageShootNumberPerPlant
    !canopyShootNumber: 576
    print *, "canopyShootNumber estimated :"
    print *, canopyShootNumber
    !leafTillerNumberArray: [1, 1, 1, 2]
    print *, "leafTillerNumberArray estimated :"
    Do i_cyml = 1, size(leafTillerNumberArray)
        print *, leafTillerNumberArray(i_cyml);
    END DO
    !tilleringProfile: [288.0, 288.0]
    print *, "tilleringProfile estimated :"
    Do i_cyml = 1, size(tilleringProfile)
        print *, tilleringProfile(i_cyml);
    END DO
    !numberTillerCohort: 2
    print *, "numberTillerCohort estimated :"
    print *, numberTillerCohort

END PROGRAM
